In [15]:
import pandas as pd
import numpy as np
import psycopg2 as pg 


In [16]:
conn = pg.connect(
    host="localhost",
    database="eeg",
    user="admin",
    password="admin123",
    port=5432
)

cur = conn.cursor()

# Import date 

In [17]:
data = pd.read_csv(r'D:\Work\DSDE\Notebook\dataset\ds004902-1.0.5\participants.tsv', sep='\t')
dataArray = data.to_numpy()

In [18]:
def checkSleep(sleep):
    if sleep == 'NS->SD':
        return 'good sleep'
    else:
        return 'bad sleep'

# insert participant's into database

In [19]:

try:
    for i in range(len(dataArray)):
        cur.execute("""INSERT INTO participant (participant_id,gender,age,session_order,sleep_status) VALUES (%s, %s, %s, %s,%s)""", (dataArray[i][0], dataArray[i][1], dataArray[i][2], dataArray[i][3], checkSleep(dataArray[i][3])))
    conn.commit()
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()


# definde Function insert Details

In [20]:
def insert_details(id, participant_id):
    cur = conn.cursor()

    for i in range(len(datadetailsArray)):
        channelName = datadetailsArray[i][0].lower()
        x = datadetailsArray[i][1]
        y = datadetailsArray[i][2]
        z = datadetailsArray[i][3]
        manufacturer = datadetailsArray[i][4]
        sqlinsert = f"INSERT INTO eegdetails (id,participant_id, x_{channelName}, y_{channelName}, z_{channelName}) VALUES ('{id}', '{participant_id}', {x}, {y}, {z})"
        sqlupdate = f"UPDATE eegdetails SET x_{channelName} = {x}, y_{channelName} = {y}, z_{channelName}  = {z} WHERE participant_id = '{participant_id}' and id = '{id}'"
        sqlmanufacturer = f"UPDATE eegdetails SET manufacturer = '{manufacturer}'  WHERE participant_id = '{participant_id}' and id = '{id}'"
        if i == 0:
            cur.execute(sqlinsert)
            conn.commit()
        elif i == len(datadetailsArray)-1:
            cur.execute(sqlmanufacturer)
            cur.execute(sqlupdate)
            conn.commit()
        else:
            cur.execute(sqlupdate)
            conn.commit()

    cur.close()

In [21]:
dataParticipant = pd.read_sql_query("SELECT id,participant_id FROM participant", conn)


dataParticipantArray = dataParticipant.to_numpy()

for i in range(len(dataParticipantArray)):
    participant_id = dataParticipantArray[i][1]
    filepath = rf"D:\Work\DSDE\Notebook\dataset\ds004902-1.0.5\{participant_id}\ses-1\eeg\{participant_id}_ses-1_electrodes.tsv"
    datadetails = pd.read_csv(filepath, sep='\t')
    datadetailsArray = datadetails.to_numpy()
    insert_details(dataParticipantArray[i][0], participant_id)


C:\Users\koonf\AppData\Local\Temp\ipykernel_31040\2559608763.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataParticipant = pd.read_sql_query("SELECT id,participant_id FROM participant", conn)


In [22]:
showData = pd.read_sql_query("SELECT * FROM participant;", conn)

showData

C:\Users\koonf\AppData\Local\Temp\ipykernel_31040\91266679.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  showData = pd.read_sql_query("SELECT * FROM participant;", conn)


,id,participant_id,gender,age,session_order,sleep_status
0,1,sub-01,M,22,NS->SD,good sleep
1,2,sub-02,M,21,NS->SD,good sleep
2,3,sub-03,F,19,NS->SD,good sleep
3,4,sub-04,M,22,NS->SD,good sleep
4,5,sub-05,F,18,NS->SD,good sleep
...,...,...,...,...,...,...
66,67,sub-67,F,21,SD->NS,bad sleep
67,68,sub-68,F,21,SD->NS,bad sleep
68,69,sub-69,F,18,NS->SD,good sleep
69,70,sub-70,M,19,NS->SD,good sleep
